In [1]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-50gwddmr
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-50gwddmr


In [5]:
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [6]:
%%cuda --name TimingGPU.h

#ifndef __TIMINGGPU_H__
#define __TIMINGGPU_H__

/**************/
/* TIMING GPU */
/**************/

// Events are a part of CUDA API and provide a system independent way to measure execution times on CUDA devices with approximately 0.5
// microsecond precision.

struct PrivateTimingGPU;

class TimingGPU
{
private:
	PrivateTimingGPU *privateTimingGPU;

public:

	TimingGPU();

	~TimingGPU();

	void StartCounter();
	void StartCounterFlags();

	float GetCounter();

}; // TimingCPU class

#endif

'File written in /content/src/TimingGPU.h'

In [7]:
%%cuda --name Utilities.h

#ifndef UTILITIES_H
#define UTILITIES_H

#include <cusolverDn.h>
#include <cublas_v2.h>
#include <cusparse_v2.h>
#include <curand.h>
#include <cufft.h>

//#include <thrust/pair.h>

//extern "C" int iDivUp(int, int);
__host__ __device__ int iDivUp(int, int);
extern "C" void gpuErrchk(cudaError_t);
extern "C" void cusolveSafeCall(cusolverStatus_t);
extern "C" void cublasSafeCall(cublasStatus_t);
extern "C" void cufftSafeCall(cufftResult err);
extern "C" void cusparseSafeCall(cusparseStatus_t err);
extern "C" void curandSafeCall(curandStatus_t err);

template <class T>
void reverseArray(const T * __restrict__, T * __restrict__, const int, const T a = static_cast<T>(1));

//template <class T>
//thrust::pair<T *,T *> Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a = static_cast<T>(1));
//
//template <class T>
//thrust::pair<T *,T *> h_Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a = static_cast<T>(1));

template<class T>
T h_l2_norm(T *v1, T *v2, const int N);

template <class T>
void linearCombination(const T * __restrict__, const T * __restrict__, T * __restrict__, const int, const int, const cublasHandle_t);

void linearCombination(const float * __restrict__, const float * __restrict__, float * __restrict__,
	const int, const int, const cublasHandle_t);

void linearCombination(const double * __restrict__, const double * __restrict__, double * __restrict__,
	const int, const int, const cublasHandle_t);

template<class T>
void vectorAddConstant(T * __restrict__, const T, const int);

template<class T>
void vectorMulConstant(T * __restrict__, const T, const int);

template<class T>
void h_vectorMulConstant(T * __restrict__, const T, const int);

template<class T>
__host__ __device__ T fma2(T, T, T);

__device__ int modulo(int, int);

__device__ double atomicAdd(double *, double);
__device__ float  atomicMin(float *, float);

double deg2rad(double);

void cudaMemoryUsage();

/**************************/
/* TEMPLATE SHARED MEMORY */
/**************************/
// --- Credit to the simpleTemplates CUDA sample
template <typename T>
struct SharedMemory
{
	// Ensure that we won't compile any un-specialized types
	__device__ T *getPointer()
	{
		extern __device__ void error(void);
		error();
		return NULL;
	}
};

// Following are the specializations for the following types.
// int, uint, char, uchar, short, ushort, long, ulong, bool, float, and double
// One could also specialize it for user-defined types.

template <>
struct SharedMemory <int>
{
	__device__ int *getPointer()
	{
		extern __shared__ int s_int[];
		return s_int;
	}
};

template <>
struct SharedMemory <unsigned int>
{
	__device__ unsigned int *getPointer()
	{
		extern __shared__ unsigned int s_uint[];
		return s_uint;
	}
};

template <>
struct SharedMemory <char>
{
	__device__ char *getPointer()
	{
		extern __shared__ char s_char[];
		return s_char;
	}
};

template <>
struct SharedMemory <unsigned char>
{
	__device__ unsigned char *getPointer()
	{
		extern __shared__ unsigned char s_uchar[];
		return s_uchar;
	}
};

template <>
struct SharedMemory <short>
{
	__device__ short *getPointer()
	{
		extern __shared__ short s_short[];
		return s_short;
	}
};

template <>
struct SharedMemory <unsigned short>
{
	__device__ unsigned short *getPointer()
	{
		extern __shared__ unsigned short s_ushort[];
		return s_ushort;
	}
};

template <>
struct SharedMemory <long>
{
	__device__ long *getPointer()
	{
		extern __shared__ long s_long[];
		return s_long;
	}
};

template <>
struct SharedMemory <unsigned long>
{
	__device__ unsigned long *getPointer()
	{
		extern __shared__ unsigned long s_ulong[];
		return s_ulong;
	}
};

template <>
struct SharedMemory <bool>
{
	__device__ bool *getPointer()
	{
		extern __shared__ bool s_bool[];
		return s_bool;
	}
};

template <>
struct SharedMemory <float>
{
	__device__ float *getPointer()
	{
		extern __shared__ float s_float[];
		return s_float;
	}
};

template <>
struct SharedMemory <double>
{
	__device__ double *getPointer()
	{
		extern __shared__ double s_double[];
		return s_double;
	}
};

#endif

'File written in /content/src/Utilities.h'

In [8]:
%%cuda --name Utilities.cu

#include <stdio.h>
#include <assert.h>
//#include <math.h>

#include "cuda_runtime.h"
#include <cuda.h>

#include <cusolverDn.h>
#include <cublas_v2.h>
#include <cufft.h>

#include "Utilities.h"

#define DEBUG

#define PI_R         3.14159265358979323846f

/*******************/
/* iDivUp FUNCTION */
/*******************/
//extern "C" int iDivUp(int a, int b){ return ((a % b) != 0) ? (a / b + 1) : (a / b); }
__host__ __device__ int iDivUp(int a, int b) { return ((a % b) != 0) ? (a / b + 1) : (a / b); }

/********************/
/* CUDA ERROR CHECK */
/********************/
// --- Credit to http://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true)
{
	if (code != cudaSuccess)
	{
		fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
		if (abort) { exit(code); }
	}
}

extern "C" void gpuErrchk(cudaError_t ans) { gpuAssert((ans), __FILE__, __LINE__); }

/**************************/
/* CUSOLVE ERROR CHECKING */
/**************************/
static const char *_cusolverGetErrorEnum(cusolverStatus_t error)
{
	switch (error)
	{
	case CUSOLVER_STATUS_SUCCESS:
		return "CUSOLVER_SUCCESS";

	case CUSOLVER_STATUS_NOT_INITIALIZED:
		return "CUSOLVER_STATUS_NOT_INITIALIZED";

	case CUSOLVER_STATUS_ALLOC_FAILED:
		return "CUSOLVER_STATUS_ALLOC_FAILED";

	case CUSOLVER_STATUS_INVALID_VALUE:
		return "CUSOLVER_STATUS_INVALID_VALUE";

	case CUSOLVER_STATUS_ARCH_MISMATCH:
		return "CUSOLVER_STATUS_ARCH_MISMATCH";

	case CUSOLVER_STATUS_EXECUTION_FAILED:
		return "CUSOLVER_STATUS_EXECUTION_FAILED";

	case CUSOLVER_STATUS_INTERNAL_ERROR:
		return "CUSOLVER_STATUS_INTERNAL_ERROR";

	case CUSOLVER_STATUS_MATRIX_TYPE_NOT_SUPPORTED:
		return "CUSOLVER_STATUS_MATRIX_TYPE_NOT_SUPPORTED";

	}

	return "<unknown>";
}

inline void __cusolveSafeCall(cusolverStatus_t err, const char *file, const int line)
{
	if (CUSOLVER_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUSOLVE error in file '%s', line %d, error: %s \nterminating!\n", __FILE__, __LINE__, \
			_cusolverGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cusolveSafeCall(cusolverStatus_t err) { __cusolveSafeCall(err, __FILE__, __LINE__); }

/*************************/
/* CUBLAS ERROR CHECKING */
/*************************/
static const char *_cublasGetErrorEnum(cublasStatus_t error)
{
	switch (error)
	{
	case CUBLAS_STATUS_SUCCESS:
		return "CUBLAS_STATUS_SUCCESS";

	case CUBLAS_STATUS_NOT_INITIALIZED:
		return "CUBLAS_STATUS_NOT_INITIALIZED";

	case CUBLAS_STATUS_ALLOC_FAILED:
		return "CUBLAS_STATUS_ALLOC_FAILED";

	case CUBLAS_STATUS_INVALID_VALUE:
		return "CUBLAS_STATUS_INVALID_VALUE";

	case CUBLAS_STATUS_ARCH_MISMATCH:
		return "CUBLAS_STATUS_ARCH_MISMATCH";

	case CUBLAS_STATUS_MAPPING_ERROR:
		return "CUBLAS_STATUS_MAPPING_ERROR";

	case CUBLAS_STATUS_EXECUTION_FAILED:
		return "CUBLAS_STATUS_EXECUTION_FAILED";

	case CUBLAS_STATUS_INTERNAL_ERROR:
		return "CUBLAS_STATUS_INTERNAL_ERROR";

	case CUBLAS_STATUS_NOT_SUPPORTED:
		return "CUBLAS_STATUS_NOT_SUPPORTED";

	case CUBLAS_STATUS_LICENSE_ERROR:
		return "CUBLAS_STATUS_LICENSE_ERROR";
	}

	return "<unknown>";
}

inline void __cublasSafeCall(cublasStatus_t err, const char *file, const int line)
{
	if (CUBLAS_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUBLAS error in file '%s', line %d, error: %s\nterminating!\n", __FILE__, __LINE__, \
			_cublasGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cublasSafeCall(cublasStatus_t err) { __cublasSafeCall(err, __FILE__, __LINE__); }

/************************/
/* CUFFT ERROR CHECKING */
/************************/
// See http://stackoverflow.com/questions/16267149/cufft-error-handling
static const char *_cudaGetErrorEnum(cufftResult error)
{
	switch (error)
	{
	case CUFFT_SUCCESS:
		return "CUFFT_SUCCESS - The cuFFT operation was successful";

	case CUFFT_INVALID_PLAN:
		return "CUFFT_INVALID_PLAN - cuFFT was passed an invalid plan handle";

	case CUFFT_ALLOC_FAILED:
		return "CUFFT_ALLOC_FAILED - cuFFT failed to allocate GPU or CPU memory";

	case CUFFT_INVALID_TYPE:
		return "CUFFT_INVALID_TYPE - No longer used";

	case CUFFT_INVALID_VALUE:
		return "CUFFT_INVALID_VALUE - User specified an invalid pointer or parameter";

	case CUFFT_INTERNAL_ERROR:
		return "CUFFT_INTERNAL_ERROR - Driver or internal cuFFT library error";

	case CUFFT_EXEC_FAILED:
		return "CUFFT_EXEC_FAILED - Failed to execute an FFT on the GPU";

	case CUFFT_SETUP_FAILED:
		return "CUFFT_SETUP_FAILED - The cuFFT library failed to initialize";

	case CUFFT_INVALID_SIZE:
		return "CUFFT_INVALID_SIZE - User specified an invalid transform size";

	case CUFFT_UNALIGNED_DATA:
		return "CUFFT_UNALIGNED_DATA - No longer used";

	case CUFFT_INCOMPLETE_PARAMETER_LIST:
		return "CUFFT_INCOMPLETE_PARAMETER_LIST - Missing parameters in call";

	case CUFFT_INVALID_DEVICE:
		return "CUFFT_INVALID_DEVICE - Execution of a plan was on different GPU than plan creation";

	case CUFFT_PARSE_ERROR:
		return "CUFFT_PARSE_ERROR - Internal plan database error";

	case CUFFT_NO_WORKSPACE:
		return "CUFFT_NO_WORKSPACE - No workspace has been provided prior to plan execution";

	case CUFFT_NOT_IMPLEMENTED:
		return "CUFFT_NOT_IMPLEMENTED - Function does not implement functionality for parameters given";

	case CUFFT_LICENSE_ERROR:
		return "CUFFT_LICENSE_ERROR - Used in previous versions";

	case CUFFT_NOT_SUPPORTED:
		return "CUFFT_NOT_SUPPORTED - Operation is not supported for parameters given";
	}

	return "<unknown>";
}

// --- CUFFTSAFECALL
inline void cufftAssert(cufftResult err, const char *file, const int line, bool abort = true)
{
	if (CUFFT_SUCCESS != err) {
		fprintf(stderr, "CUFFTassert: Error nr. %d - %s %s %d\n", err, _cudaGetErrorEnum(err), __FILE__, __LINE__);
		if (abort) exit(err);
	}
}

extern "C" void cufftSafeCall(cufftResult err) { cufftAssert(err, __FILE__, __LINE__); }

/***************************/
/* CUSPARSE ERROR CHECKING */
/***************************/
static const char *_cusparseGetErrorEnum(cusparseStatus_t error)
{
	switch (error)
	{

	case CUSPARSE_STATUS_SUCCESS:
		return "CUSPARSE_STATUS_SUCCESS";

	case CUSPARSE_STATUS_NOT_INITIALIZED:
		return "CUSPARSE_STATUS_NOT_INITIALIZED";

	case CUSPARSE_STATUS_ALLOC_FAILED:
		return "CUSPARSE_STATUS_ALLOC_FAILED";

	case CUSPARSE_STATUS_INVALID_VALUE:
		return "CUSPARSE_STATUS_INVALID_VALUE";

	case CUSPARSE_STATUS_ARCH_MISMATCH:
		return "CUSPARSE_STATUS_ARCH_MISMATCH";

	case CUSPARSE_STATUS_MAPPING_ERROR:
		return "CUSPARSE_STATUS_MAPPING_ERROR";

	case CUSPARSE_STATUS_EXECUTION_FAILED:
		return "CUSPARSE_STATUS_EXECUTION_FAILED";

	case CUSPARSE_STATUS_INTERNAL_ERROR:
		return "CUSPARSE_STATUS_INTERNAL_ERROR";

	case CUSPARSE_STATUS_MATRIX_TYPE_NOT_SUPPORTED:
		return "CUSPARSE_STATUS_MATRIX_TYPE_NOT_SUPPORTED";

	case CUSPARSE_STATUS_ZERO_PIVOT:
		return "CUSPARSE_STATUS_ZERO_PIVOT";
	}

	return "<unknown>";
}

inline void __cusparseSafeCall(cusparseStatus_t err, const char *file, const int line)
{
	if (CUSPARSE_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUSPARSE error in file '%s', line %d, error %s\nterminating!\n", __FILE__, __LINE__, \
			_cusparseGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cusparseSafeCall(cusparseStatus_t err) { __cusparseSafeCall(err, __FILE__, __LINE__); }

/*************************/
/* CURAND ERROR CHECKING */
/*************************/
static const char *_curandGetErrorEnum(curandStatus_t error)
{
	switch (error)
	{
	case CURAND_STATUS_SUCCESS:
		return "CURAND_SUCCESS";

	case CURAND_STATUS_VERSION_MISMATCH:
		return "CURAND_STATUS_VERSION_MISMATCH";

	case CURAND_STATUS_NOT_INITIALIZED:
		return "CURAND_STATUS_NOT_INITIALIZED";

	case CURAND_STATUS_ALLOCATION_FAILED:
		return "CURAND_STATUS_ALLOCATION_FAILED";

	case CURAND_STATUS_TYPE_ERROR:
		return "CURAND_STATUS_TYPE_ERROR";

	case CURAND_STATUS_OUT_OF_RANGE:
		return "CURAND_STATUS_OUT_OF_RANGE";

	case CURAND_STATUS_LENGTH_NOT_MULTIPLE:
		return "CURAND_STATUS_LENGTH_NOT_MULTIPLE";

	case CURAND_STATUS_DOUBLE_PRECISION_REQUIRED:
		return "CURAND_STATUS_DOUBLE_PRECISION_REQUIRED";

	case CURAND_STATUS_LAUNCH_FAILURE:
		return "CURAND_STATUS_LAUNCH_FAILURE";

	case CURAND_STATUS_PREEXISTING_FAILURE:
		return "CURAND_STATUS_PREEXISTING_FAILURE";

	case CURAND_STATUS_INITIALIZATION_FAILED:
		return "CURAND_STATUS_INITIALIZATION_FAILED";

	case CURAND_STATUS_ARCH_MISMATCH:
		return "CURAND_STATUS_ARCH_MISMATCH";

	case CURAND_STATUS_INTERNAL_ERROR:
		return "CURAND_STATUS_INTERNAL_ERROR";

	}

	return "<unknown>";
}

inline void __curandSafeCall(curandStatus_t err, const char *file, const int line)
{
	if (CURAND_STATUS_SUCCESS != err) {
		fprintf(stderr, "CURAND error in file '%s', line %d, error: %s \nterminating!\n", __FILE__, __LINE__, \
			_curandGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void curandSafeCall(curandStatus_t err) { __curandSafeCall(err, __FILE__, __LINE__); }

/************************/
/* REVERSE ARRAY KERNEL */
/************************/
#define BLOCKSIZE_REVERSE	256

// --- Credit to http://www.drdobbs.com/parallel/cuda-supercomputing-for-the-masses-part/208801731?pgno=2
template <class T>
__global__ void reverseArrayKernel(const T * __restrict__ d_in, T * __restrict__ d_out, const int N, const T a)
{
	// --- Credit to the simpleTemplates CUDA sample
	SharedMemory<T> smem;
	T* s_data = smem.getPointer();

	const int tid = blockDim.x * blockIdx.x + threadIdx.x;
	const int id = threadIdx.x;
	const int offset = blockDim.x * (blockIdx.x + 1);

	// --- Load one element per thread from device memory and store it *in reversed order* into shared memory
	if (tid < N) s_data[BLOCKSIZE_REVERSE - (id + 1)] = a * d_in[tid];

	// --- Block until all threads in the block have written their data to shared memory
	__syncthreads();

	// --- Write the data from shared memory in forward order
	if ((N - offset + id) >= 0) d_out[N - offset + id] = s_data[threadIdx.x];
}

/************************/
/* REVERSE ARRAY KERNEL */
/************************/
template <class T>
void reverseArray(const T * __restrict__ d_in, T * __restrict__ d_out, const int N, const T a) {

	reverseArrayKernel << <iDivUp(N, BLOCKSIZE_REVERSE), BLOCKSIZE_REVERSE, BLOCKSIZE_REVERSE * sizeof(T) >> >(d_in, d_out, N, a);
#ifdef DEBUG
	gpuErrchk(cudaPeekAtLastError());
	gpuErrchk(cudaDeviceSynchronize());
#endif

}

template void reverseArray<float>(const float  * __restrict__, float  * __restrict__, const int, const float);
template void reverseArray<double>(const double * __restrict__, double * __restrict__, const int, const double);

/********************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION KERNEL */
/********************************************************/
#define BLOCKSIZE_CART2POL	256

template <class T>
__global__ void Cartesian2PolarKernel(const T * __restrict__ d_x, const T * __restrict__ d_y, T * __restrict__ d_rho, T * __restrict__ d_theta,
	const int N, const T a) {

	const int tid = blockIdx.x * blockDim.x + threadIdx.x;

	if (tid < N) {
		d_rho[tid] = a * hypot(d_x[tid], d_y[tid]);
		d_theta[tid] = atan2(d_y[tid], d_x[tid]);
	}

}

/*******************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION - GPU */
/*******************************************************/
//template <class T>
//thrust::pair<T *,T *> Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a) {
//
//	T *d_rho;	gpuErrchk(cudaMalloc((void**)&d_rho,   N * sizeof(T)));
//	T *d_theta; gpuErrchk(cudaMalloc((void**)&d_theta, N * sizeof(T)));
//
//	Cartesian2PolarKernel<<<iDivUp(N, BLOCKSIZE_CART2POL), BLOCKSIZE_CART2POL>>>(d_x, d_y, d_rho, d_theta, N, a);
//#ifdef DEBUG
//	gpuErrchk(cudaPeekAtLastError());
//	gpuErrchk(cudaDeviceSynchronize());
//#endif
//
//	return thrust::make_pair(d_rho, d_theta);
//}
//
//template thrust::pair<float  *, float  *>  Cartesian2Polar<float>  (const float  *, const float  *, const int, const float);
//template thrust::pair<double *, double *>  Cartesian2Polar<double> (const double *, const double *, const int, const double);

/*******************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION - CPU */
/*******************************************************/
//template <class T>
//thrust::pair<T *,T *> h_Cartesian2Polar(const T * __restrict__ h_x, const T * __restrict__ h_y, const int N, const T a) {
//
//	T *h_rho	= (T *)malloc(N * sizeof(T));
//	T *h_theta	= (T *)malloc(N * sizeof(T));
//
//	for (int i = 0; i < N; i++) {
//		h_rho[i]	= a * hypot(h_x[i], h_y[i]);
//		h_theta[i]	= atan2(h_y[i], h_x[i]);
//	}
//
//	return thrust::make_pair(h_rho, h_theta);
//}
//
//template thrust::pair<float  *, float  *>  h_Cartesian2Polar<float>  (const float  *, const float  *, const int, const float);
//template thrust::pair<double *, double *>  h_Cartesian2Polar<double> (const double *, const double *, const int, const double);

/*******************************/
/* COMPUTE L2 NORM OF A VECTOR */
/*******************************/
template<class T>
T h_l2_norm(T *v1, T *v2, const int N) {

	T norm = (T)0;

	for (int i = 0; i < N; ++i)
	{
		T d = v1[i] - v2[i];
		norm = norm + d * d;
	}

	return sqrt(norm);
}

template float  h_l2_norm<float>(float  *, float  *, const int);
template double h_l2_norm<double>(double *, double *, const int);

/*******************************/
/* LINEAR COMBINATION FUNCTION */
/*******************************/
void linearCombination(const float * __restrict__ d_coeff, const float * __restrict__ d_basis_functions_real, float * __restrict__ d_linear_combination,
	const int N_basis_functions, const int N_sampling_points, const cublasHandle_t handle) {

	float alpha = 1.f;
	float beta = 0.f;
	cublasSafeCall(cublasSgemv(handle, CUBLAS_OP_N, N_sampling_points, N_basis_functions, &alpha, d_basis_functions_real, N_sampling_points,
		d_coeff, 1, &beta, d_linear_combination, 1));

}

void linearCombination(const double * __restrict__ d_coeff, const double * __restrict__ d_basis_functions_real, double * __restrict__ d_linear_combination,
	const int N_basis_functions, const int N_sampling_points, const cublasHandle_t handle) {

	double alpha = 1.;
	double beta = 0.;
	cublasSafeCall(cublasDgemv(handle, CUBLAS_OP_N, N_sampling_points, N_basis_functions, &alpha, d_basis_functions_real, N_sampling_points,
		d_coeff, 1, &beta, d_linear_combination, 1));

}

/******************************/
/* ADD A CONSTANT TO A VECTOR */
/******************************/
#define BLOCKSIZE_VECTORADDCONSTANT	256

template<class T>
__global__ void vectorAddConstantKernel(T * __restrict__ d_in, const T scalar, const int N) {

	const int tid = threadIdx.x + blockIdx.x*blockDim.x;

	if (tid < N) d_in[tid] += scalar;

}

template<class T>
void vectorAddConstant(T * __restrict__ d_in, const T scalar, const int N) {

	vectorAddConstantKernel << <iDivUp(N, BLOCKSIZE_VECTORADDCONSTANT), BLOCKSIZE_VECTORADDCONSTANT >> >(d_in, scalar, N);

}

template void  vectorAddConstant<float>(float  * __restrict__, const float, const int);
template void  vectorAddConstant<double>(double * __restrict__, const double, const int);

/*****************************************/
/* MULTIPLY A VECTOR BY A CONSTANT - GPU */
/*****************************************/
#define BLOCKSIZE_VECTORMULCONSTANT	256

template<class T>
__global__ void vectorMulConstantKernel(T * __restrict__ d_in, const T scalar, const int N) {

	const int tid = threadIdx.x + blockIdx.x*blockDim.x;

	if (tid < N) d_in[tid] *= scalar;

}

template<class T>
void vectorMulConstant(T * __restrict__ d_in, const T scalar, const int N) {

	vectorMulConstantKernel << <iDivUp(N, BLOCKSIZE_VECTORMULCONSTANT), BLOCKSIZE_VECTORMULCONSTANT >> >(d_in, scalar, N);

}

template void  vectorMulConstant<float>(float  * __restrict__, const float, const int);
template void  vectorMulConstant<double>(double * __restrict__, const double, const int);

/*****************************************/
/* MULTIPLY A VECTOR BY A CONSTANT - CPU */
/*****************************************/
template<class T>
void h_vectorMulConstant(T * __restrict__ h_in, const T scalar, const int N) {

	for (int i = 0; i < N; i++) h_in[i] *= scalar;

}

template void  h_vectorMulConstant<float>(float  * __restrict__, const float, const int);
template void  h_vectorMulConstant<double>(double * __restrict__, const double, const int);

/*****************************************************/
/* FUSED MULTIPLY ADD OPERATIONS FOR HOST AND DEVICE */
/*****************************************************/
template<class T>
__host__ __device__ T fma2(T x, T y, T z) { return x * y + z; }

template float  fma2<float >(float, float, float);
template double fma2<double>(double, double, double);

/*******************/
/* MODULO FUNCTION */
/*******************/
__device__ int modulo(int val, int _mod)
{
	int P;
	if (val > 0) { (!(_mod & (_mod - 1)) ? P = val&(_mod - 1) : P = val % (_mod)); return P; }
	else
	{
		(!(_mod & (_mod - 1)) ? P = (-val)&(_mod - 1) : P = (-val) % (_mod));
		if (P > 0) return _mod - P;
		else return 0;
	}
}

/***************************************/
/* ATOMIC ADDITION FUNCTION ON DOUBLES */
/***************************************/
#if !defined(__CUDA_ARCH__) || __CUDA_ARCH__ >= 600
#else
__device__ double atomicAdd(double* address, double val)
{
	unsigned long long int* address_as_ull =
		(unsigned long long int*)address;
	register unsigned long long int old = *address_as_ull, assumed;
	do {
		assumed = old;
		old = atomicCAS(address_as_ull, assumed,
			__double_as_longlong(val +
				__longlong_as_double(assumed)));
	} while (assumed != old);
	return __longlong_as_double(old);
}
#endif

/*********************************/
/* ATOMIC MIN FUNCTION ON FLOATS */
/*********************************/
__device__ float atomicMin(float* address, float val)
{
	int* address_as_i = (int*)address;
	int old = *address_as_i, assumed;
	do {
		assumed = old;
		old = ::atomicCAS(address_as_i, assumed,
			__float_as_int(::fminf(val, __int_as_float(assumed))));
	} while (assumed != old);
	return __int_as_float(old);
}

/*********************/
/* DEGREE TO RADIANS */
/*********************/
double deg2rad(double deg) { return deg*PI_R / 180; }

/*********************/
/* CUDA MEMORY USAGE */
/*********************/
void cudaMemoryUsage() {

	size_t free_byte;
	size_t total_byte;

	gpuErrchk(cudaMemGetInfo(&free_byte, &total_byte));

	double free_db = (double)free_byte;
	double total_db = (double)total_byte;
	double used_db = total_db - free_db;

	printf("GPU memory: used = %f, free = %f MB, total available = %f MB\n", used_db / 1024.0 / 1024.0, free_db / 1024.0 / 1024.0, total_db / 1024.0 / 1024.0);

}

'File written in /content/src/Utilities.cu'

In [9]:
%%cuda --name TimingGPU.cu

/**************/
/* TIMING GPU */
/**************/

#include "TimingGPU.h"

#include <cuda.h>
#include <cuda_runtime.h>

struct PrivateTimingGPU {
	cudaEvent_t		start;
	cudaEvent_t		stop;
};

// default constructor
TimingGPU::TimingGPU() { privateTimingGPU = new PrivateTimingGPU; }

// default destructor
TimingGPU::~TimingGPU() { }

void TimingGPU::StartCounter()
{
	cudaEventCreate(&((*privateTimingGPU).start));
	cudaEventCreate(&((*privateTimingGPU).stop));
	cudaEventRecord((*privateTimingGPU).start, 0);
}

void TimingGPU::StartCounterFlags()
{
	int eventflags = cudaEventBlockingSync;

	cudaEventCreateWithFlags(&((*privateTimingGPU).start), eventflags);
	cudaEventCreateWithFlags(&((*privateTimingGPU).stop), eventflags);
	cudaEventRecord((*privateTimingGPU).start, 0);
}

// Gets the counter in ms
float TimingGPU::GetCounter()
{
	float	time;
	cudaEventRecord((*privateTimingGPU).stop, 0);
	cudaEventSynchronize((*privateTimingGPU).stop);
	cudaEventElapsedTime(&time, (*privateTimingGPU).start, (*privateTimingGPU).stop);
	return time;
}

'File written in /content/src/TimingGPU.cu'

In [10]:
%%cuda --name FFTShift_1D.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <assert.h>

#include <cufft.h>
#include <cufftXt.h>

#include "TimingGPU.h"
#include "Utilities.h"

//#define DEBUG

#define BLOCKSIZE 256

/*****************************************/
/* FFTSHIFT 1D IN-PLACE MEMORY MOVEMENTS */
/*****************************************/
__global__ void fftshift_1D_inplace_memory_movements(float2 * __restrict__ d_inout, const unsigned int N)
{
	unsigned int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid < N/2)
    {
		float2 temp = d_inout[tid];
        d_inout[tid] = d_inout[tid + (N / 2)];
        d_inout[tid + (N / 2)] = temp;
    }
}

/*********************************************/
/* FFTSHIFT 1D OUT-OF-PLACE MEMORY MOVEMENTS */
/*********************************************/
__global__ void fftshift_1D_outofplace_memory_movements(const float2 * __restrict__ d_in, float2 * __restrict__ d_out, const unsigned int N)
{
	unsigned int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if (tid < N/2)
    {
        d_out[tid] = d_in[tid + (N / 2)];
        d_out[tid + (N / 2)] = d_in[tid];
    }
}

/**********************************************/
/* FFTSHIFT 1D INPLACE CHESSBOARD - VERSION 1 */
/**********************************************/
static __device__ float2 fftshift_1D_chessboard_callback_v1(void *d_in, size_t offset, void *callerInfo, void *sharedPtr) {

	float a		= (float)(1-2*((int)offset%2));

	float2	out = ((float2*)d_in)[offset];
	out.x = out.x * a;
	out.y = out.y * a;
	return out;
}

__device__ cufftCallbackLoadC fftshift_1D_chessboard_callback_v1_Ptr = fftshift_1D_chessboard_callback_v1;

/**********************************************/
/* FFTSHIFT 1D INPLACE CHESSBOARD - VERSION 2 */
/**********************************************/
__device__ float2 fftshift_1D_chessboard_callback_v2(void *d_in, size_t offset, void *callerInfo, void *sharedPtr) {

	float a = pow(-1, (offset&1));

	float2	out = ((float2*)d_in)[offset];
	out.x = out.x * a;
	out.y = out.y * a;
	return out;
}

__device__ cufftCallbackLoadC fftshift_1D_chessboard_callback_v2_Ptr = fftshift_1D_chessboard_callback_v2;

/**********************************************/
/* FFTSHIFT 1D INPLACE CHESSBOARD - VERSION 3 */
/**********************************************/
__device__ float2 fftshift_1D_chessboard_callback_v3(void *d_in, size_t offset, void *callerInfo, void *sharedPtr) {

	float2	out = ((float2*)d_in)[offset];

	if ((int)offset&1) {

		out.x = -out.x;
		out.y = -out.y;

	}
	return out;
}

__device__ cufftCallbackLoadC fftshift_1D_chessboard_callback_v3_Ptr = fftshift_1D_chessboard_callback_v3;

/********/
/* MAIN */
/********/
int main()
{
	//const int N = 131072;
	//const int N = 524288;
	const int N = 2097152;
  //const int N = 64;

	TimingGPU timerGPU;

	// --- Host side input array
	float2 *h_vect = (float2 *)malloc(N * sizeof(float2));
	for (int i = 0; i < N; i++) {
		h_vect[i].x = (float)rand() / (float)RAND_MAX;
		h_vect[i].y = (float)rand() / (float)RAND_MAX;
	}

	// --- Host side output arrays
	float2 *h_out1 = (float2 *)malloc(N * sizeof(float2));
	float2 *h_out2 = (float2 *)malloc(N * sizeof(float2));
	float2 *h_out3 = (float2 *)malloc(N * sizeof(float2));
	float2 *h_out4 = (float2 *)malloc(N * sizeof(float2));
	float2 *h_out5 = (float2 *)malloc(N * sizeof(float2));

	// --- Device side input arrays
	float2 *d_vect1; gpuErrchk(cudaMalloc(&d_vect1, N * sizeof(float2)));
	float2 *d_vect2; gpuErrchk(cudaMalloc(&d_vect2, N * sizeof(float2)));
	float2 *d_vect3; gpuErrchk(cudaMalloc(&d_vect3, N * sizeof(float2)));
	float2 *d_vect4; gpuErrchk(cudaMalloc(&d_vect4, N * sizeof(float2)));
	float2 *d_vect5; gpuErrchk(cudaMalloc(&d_vect5, N * sizeof(float2)));
 	gpuErrchk(cudaMemcpy(d_vect1, h_vect, N * sizeof(float2), cudaMemcpyHostToDevice));
	gpuErrchk(cudaMemcpy(d_vect2, h_vect, N * sizeof(float2), cudaMemcpyHostToDevice));
	gpuErrchk(cudaMemcpy(d_vect3, h_vect, N * sizeof(float2), cudaMemcpyHostToDevice));
	gpuErrchk(cudaMemcpy(d_vect4, h_vect, N * sizeof(float2), cudaMemcpyHostToDevice));
	gpuErrchk(cudaMemcpy(d_vect5, h_vect, N * sizeof(float2), cudaMemcpyHostToDevice));

	// --- Device side output arrays
	float2 *d_out1; gpuErrchk(cudaMalloc(&d_out1, N * sizeof(float2)));
	float2 *d_out2; gpuErrchk(cudaMalloc(&d_out2, N * sizeof(float2)));
	float2 *d_out3; gpuErrchk(cudaMalloc(&d_out3, N * sizeof(float2)));
	float2 *d_out4; gpuErrchk(cudaMalloc(&d_out4, N * sizeof(float2)));
	float2 *d_out5; gpuErrchk(cudaMalloc(&d_out5, N * sizeof(float2)));

	/***************************************************************/
	/* VERSION 1: cuFFT + IN-PLACE MEMORY MOVEMENTS BASED FFTSHIFT */
	/***************************************************************/
	cufftHandle planinverse; cufftSafeCall(cufftPlan1d(&planinverse, N, CUFFT_C2C, 1));
	timerGPU.StartCounter();
	cufftSafeCall(cufftExecC2C(planinverse, d_vect1, d_vect1, CUFFT_INVERSE));
	fftshift_1D_inplace_memory_movements<<<iDivUp(N/2, BLOCKSIZE), BLOCKSIZE>>>(d_vect1, N);
#ifdef DEBUG
	gpuErrchk(cudaPeekAtLastError());
	gpuErrchk(cudaDeviceSynchronize());
#endif
	printf("In-place memory movements elapsed time:  %3.3f ms \n", timerGPU.GetCounter());
	gpuErrchk(cudaMemcpy(h_out1, d_vect1, N * sizeof(float2), cudaMemcpyDeviceToHost));

	/*******************************************************************/
	/* VERSION 2: cuFFT + OUT-OF-PLACE MEMORY MOVEMENTS BASED FFTSHIFT */
	/*******************************************************************/
	cufftHandle planinverse_v1; cufftSafeCall(cufftPlan1d(&planinverse_v1, N, CUFFT_C2C, 1));
	timerGPU.StartCounter();
	cufftSafeCall(cufftExecC2C(planinverse_v1, d_vect2, d_vect2, CUFFT_INVERSE));
	fftshift_1D_outofplace_memory_movements<<<iDivUp(N/2, BLOCKSIZE), BLOCKSIZE>>>(d_vect2, d_out2, N);
#ifdef DEBUG
	gpuErrchk(cudaPeekAtLastError());
	gpuErrchk(cudaDeviceSynchronize());
#endif
	printf("Out-of-place memory movements elapsed time:  %3.3f ms \n", timerGPU.GetCounter());
	gpuErrchk(cudaMemcpy(h_out2, d_out2, N * sizeof(float2), cudaMemcpyDeviceToHost));

	// --- Checking the results
	for (int i=0; i<N; i++) if ((h_out1[i].x != h_out2[i].x)||(h_out1[i].y != h_out2[i].y)) { printf("Out-of-place memory movements test failed!\n"); return 0; }

	printf("Out-of-place memory movements test passed!\n");

	/***************************************************/
	/* VERSION 3: CHESSBOARD MULTIPLICATION V1 + cuFFT */
	/***************************************************/
	cufftCallbackLoadC hfftshift_1D_chessboard_callback_v1_Ptr;

	gpuErrchk(cudaMemcpyFromSymbol(&hfftshift_1D_chessboard_callback_v1_Ptr, fftshift_1D_chessboard_callback_v1_Ptr, sizeof(hfftshift_1D_chessboard_callback_v1_Ptr)));
	cufftHandle planinverse_v2; cufftSafeCall(cufftCreate(&planinverse_v2));
	size_t work_size_v2; cufftSafeCall(cufftMakePlan1d(planinverse_v2, N, CUFFT_C2C, 1, &work_size_v2));
	cufftSafeCall(cufftXtSetCallback(planinverse_v2, (void **)&hfftshift_1D_chessboard_callback_v1_Ptr, CUFFT_CB_LD_COMPLEX, NULL));
	timerGPU.StartCounter();
	cufftSafeCall(cufftExecC2C(planinverse_v2, d_vect3, d_out3, CUFFT_INVERSE));
	printf("Chessboard v1 elapsed time:  %3.3f ms \n", timerGPU.GetCounter());

	gpuErrchk(cudaMemcpy(h_out3, d_out3, N*sizeof(float2), cudaMemcpyDeviceToHost));

	// --- Checking the results
	for (int i=0; i<N; i++) if ((h_out1[i].x != h_out3[i].x)||(h_out1[i].y != h_out3[i].y)) { printf("Chessboard v1 test failed!\n"); return 0; }

	printf("Chessboard v1 test passed!\n");

	/****************************************/
	/* CHESSBOARD MULTIPLICATION V2 + cuFFT */
	/****************************************/
	cufftCallbackLoadC hfftshift_1D_chessboard_callback_v2_Ptr;

	gpuErrchk(cudaMemcpyFromSymbol(&hfftshift_1D_chessboard_callback_v2_Ptr, fftshift_1D_chessboard_callback_v2_Ptr, sizeof(hfftshift_1D_chessboard_callback_v2_Ptr)));
	cufftHandle planinverse_v3; cufftSafeCall(cufftCreate(&planinverse_v3));
	size_t work_size_v3; cufftSafeCall(cufftMakePlan1d(planinverse_v3, N, CUFFT_C2C, 1, &work_size_v3));
	cufftSafeCall(cufftXtSetCallback(planinverse_v3, (void **)&hfftshift_1D_chessboard_callback_v2_Ptr, CUFFT_CB_LD_COMPLEX, 0));
	timerGPU.StartCounter();
	cufftSafeCall(cufftExecC2C(planinverse_v3, (float2 *)d_vect4, (float2 *)d_out4, CUFFT_INVERSE));
	printf("Chessboard v2 elapsed time:  %3.3f ms \n", timerGPU.GetCounter());

	gpuErrchk(cudaMemcpy(h_out4, d_out4, N*sizeof(float2), cudaMemcpyDeviceToHost));

	// --- Checking the results
	for (int i=0; i<N; i++) if ((h_out1[i].x != h_out4[i].x)||(h_out1[i].y != h_out4[i].y)) { printf("Chessboard v2 test failed!\n"); return 0; }

	printf("Chessboard v2 test passed!\n");

	/****************************************/
	/* CHESSBOARD MULTIPLICATION V3 + cuFFT */
	/****************************************/
	cufftCallbackLoadC hfftshift_1D_chessboard_callback_v3_Ptr;

	gpuErrchk(cudaMemcpyFromSymbol(&hfftshift_1D_chessboard_callback_v3_Ptr, fftshift_1D_chessboard_callback_v3_Ptr, sizeof(hfftshift_1D_chessboard_callback_v3_Ptr)));
	cufftHandle planinverse_v4; cufftSafeCall(cufftCreate(&planinverse_v4));
	size_t work_size_v4; cufftSafeCall(cufftMakePlan1d(planinverse_v4, N, CUFFT_C2C, 1, &work_size_v4));
	cufftSafeCall(cufftXtSetCallback(planinverse_v4, (void **)&hfftshift_1D_chessboard_callback_v3_Ptr, CUFFT_CB_LD_COMPLEX, 0));
	timerGPU.StartCounter();
	cufftSafeCall(cufftExecC2C(planinverse_v4, d_vect5, d_out5, CUFFT_INVERSE));
	printf("Chessboard v3 elapsed time:  %3.3f ms \n", timerGPU.GetCounter());

	gpuErrchk(cudaMemcpy(h_out5, d_out5, N*sizeof(float2), cudaMemcpyDeviceToHost));

	// --- Checking the results
	for (int i=0; i<N; i++) if ((h_out1[i].x != h_out5[i].x)||(h_out1[i].y != h_out5[i].y)) { printf("Chessboard v3 test failed!\n"); return 0; }

	printf("Chessboard v3 test passed!\n");

	return 0;
}

'File written in /content/src/FFTShift_1D.cu'

In [11]:
!nvcc -arch=sm_75 -o "/content/src/FFTShift_1D.o" /content/src/FFTShift_1D.cu /content/src/TimingGPU.cu /content/src/Utilities.cu -lcufft_static -lculibos -lcublas --relocatable-device-code=true

In [12]:
!chmod 755 /content/src/FFTShift_1D.o
!/content/src/FFTShift_1D.o

In-place memory movements elapsed time:  0.713 ms 
Out-of-place memory movements elapsed time:  0.715 ms 
Out-of-place memory movements test passed!
Chessboard v1 elapsed time:  0.610 ms 
Chessboard v1 test passed!
Chessboard v2 elapsed time:  5.357 ms 
Chessboard v2 test passed!
Chessboard v3 elapsed time:  0.605 ms 
Chessboard v3 test passed!


https://github.com/zchee/cuda-sample/blob/master/7_CUDALibraries/simpleCUFFT_callback/simpleCUFFT_callback.cu